In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mmseg.apis import init_model, inference_model, show_result_pyplot
import mmcv
import cv2

# load the config file

In [5]:
# 载入 config 配置文件
from mmengine import Config
cfg = Config.fromfile('config_files/hrnet-AuroraDataset_fcn.py')

In [ ]:
from mmengine.runner import Runner
from mmseg.utils import register_all_modules

# register all modules in mmseg into the registries
# do not init the default scope here because it will be init in the runner

register_all_modules(init_default_scope=False)
runner = Runner.from_cfg(cfg)

# load the model

In [7]:
checkpoint_path = './work_dirs/AuroraDataset/hrnet-fcn20/iter_3000.pth'
model = init_model(cfg, checkpoint_path, 'cuda:0')

Loads checkpoint by local backend from path: ./work_dirs/AuroraDataset/hrnet-fcn20/iter_3000.pth


# inference

In [ ]:
from PIL import Image
import shutil
src_root = 'D://research/OnsetStatistics/data/isolated_jpgs3'
file_list = os.listdir(src_root)
file_list.sort()
tmp_root = 'data/Aurora-dataset/img_dir/test'
dst_root = 'D://research/OnsetStatistics/data/isolated_pures3_hrseg2'

for file in file_list:
    print(file)
    img_list = os.listdir(os.path.join(src_root, file))
    img_list.sort()
    
    if not os.path.exists(os.path.join(dst_root, file)):
        os.makedirs(os.path.join(dst_root, file))
    
    for im in img_list:
        # print(im)
        shutil.copyfile(os.path.join(src_root, file, im), os.path.join(tmp_root, im))
        # print(os.path.join(src_root, file, im))
        img = mmcv.imread(os.path.join(tmp_root, im))
        result = inference_model(model, img)
        pred_mask = result.pred_sem_seg.data[0].cpu().numpy()
        # print(pred_mask.shape)
        mask_img = Image.fromarray(pred_mask.astype(np.uint8)*255)
        mask_img.save(os.path.join(dst_root, file, im))
        os.remove(os.path.join(tmp_root, im))